In [ ]:
import urllib.request
import pandas as pd
import json
from pandas import json_normalize

In [48]:
try:
    url = "https://gateway.apiportal.ns.nl/reisinformatie-api/api/v2/stations?countryCodes=nl"

    hdr = {
        # Request headers
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': '24d2523a1cbc46048000f3fb3105bc0d',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)

    if response.getcode() == 200:
        data = json.loads(response.read())

        # Assuming the relevant data is in the 'payload' key, adjust this according to the actual structure of the API response
        api_data = data.get('payload', [])

        # Convert the data into a DataFrame
        stations_df = pd.json_normalize(api_data)

        # Display the DataFrame
        print(stations_df)
    else:
        print(f"Error: {response.getcode()} - {response.reason}")

except Exception as e:
    print(e)



     UICCode          stationType  EVACode  code  cdCode  \
0    8400301  FACULTATIEF_STATION  8400020   HRY     301   
1    8400534  FACULTATIEF_STATION  8400015  RTST     534   
2    8400058         MEGA_STATION  8400058   ASD      58   
3    8400282         MEGA_STATION  8400282   GVC     282   
4    8400206         MEGA_STATION  8400206   EHV     206   
..       ...                  ...      ...   ...     ...   
392  8400737    STOPTREIN_STATION  8400737   ZVB     737   
393  8400740    STOPTREIN_STATION  8400740  ZTMO     740   
394  8400743    STOPTREIN_STATION  8400743    ZH     743   
395  8400752    STOPTREIN_STATION  8400752   ZWD     752   
396  8400746    STOPTREIN_STATION  8400746  ZLSH     746   

                                                sporen  \
0         [{'spoorNummer': '1'}, {'spoorNummer': '2'}]   
1    [{'spoorNummer': '3a'}, {'spoorNummer': '3b'},...   
2    [{'spoorNummer': '1'}, {'spoorNummer': '2'}, {...   
3    [{'spoorNummer': '1'}, {'spoorNummer': '2'

In [49]:
stations_df.to_csv('stations.csv',index=False)

In [111]:
try:
    url = "https://gateway.apiportal.ns.nl/reisinformatie-api/api/v3/disruptions?isActive=false"

    hdr = {
        # Request headers
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': '24d2523a1cbc46048000f3fb3105bc0d',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)

    if response.getcode() == 200:
        data = json.loads(response.read())

        # Assuming the response is a list
        # Convert the data into a DataFrame
        df_d = pd.DataFrame(data)
        df_d = pd.json_normalize(data)

        # Display the DataFrame
        print(df_d)
    else:
        print(f"Error: {response.getcode()} - {response.reason}")

except Exception as e:
    print(e)


         id         type          registrationTime               releaseTime  \
0   6027192   DISRUPTION  2023-11-25T15:40:10+0100  2023-11-25T15:40:10+0100   
1   6027193   DISRUPTION  2023-11-25T18:22:52+0100  2023-11-25T18:22:52+0100   
2   6027194   DISRUPTION  2023-11-25T20:11:44+0100  2023-11-25T20:11:44+0100   
3   7004120  MAINTENANCE  2023-11-03T11:56:03+0100  2023-11-03T11:56:03+0100   
4   7004222  MAINTENANCE  2023-11-17T20:59:20+0100  2023-11-17T20:59:20+0100   
5   7004221  MAINTENANCE  2023-11-16T14:49:31+0100  2023-11-16T14:49:31+0100   
6   7004069  MAINTENANCE  2023-10-30T10:09:33+0100  2023-10-30T10:09:33+0100   
7   7004109  MAINTENANCE  2023-10-30T09:33:51+0100  2023-10-30T09:33:51+0100   
8   7004104  MAINTENANCE  2023-11-14T09:23:09+0100  2023-11-14T09:23:09+0100   
9   7004164  MAINTENANCE  2023-10-27T14:09:21+0200  2023-10-27T14:09:21+0200   
10  7003873  MAINTENANCE  2023-11-20T20:43:29+0100  2023-11-20T20:43:29+0100   
11  7004031  MAINTENANCE  2023-11-25T09:

In [ ]:
df_d.info()

In [115]:
df_unnested2 = pd.json_normalize(df_d['publicationSections'].explode())

In [118]:
df_unnested3=pd.json_normalize(df_d['section.stations'].explode())

In [116]:
# Combine the unnested data with the original DataFrame
df_d = pd.concat([df_d, df_unnested2], axis=1)

In [119]:
df_d=pd.concat([df_d,df_unnested3],axis=1)

In [122]:
df_d1=df_d.drop(['titleSections','consequence.section.stations'],axis=1)

In [126]:
df_d1.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 716 entries, 0 to 715
Data columns (total 35 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    48 non-null     object 
 1   type                                                  48 non-null     object 
 2   registrationTime                                      48 non-null     object 
 3   releaseTime                                           48 non-null     object 
 4   local                                                 48 non-null     object 
 5   title                                                 48 non-null     object 
 6   topic                                                 3 non-null      object 
 7   isActive                                              48 non-null     object 
 8   start                                                 48 non

In [127]:
df_d2=df_d1.drop(['publicationSections','timespans','alternativeTransportTimespans'], axis=1)

In [133]:
df_d2.columns

Index(['id', 'type', 'registrationTime', 'releaseTime', 'local', 'title',
       'topic', 'isActive', 'start', 'end', 'phase.id', 'phase.label',
       'impact.value', 'expectedDuration.description',
       'expectedDuration.endTime', 'period',
       'summaryAdditionalTravelTime.label',
       'summaryAdditionalTravelTime.shortLabel',
       'summaryAdditionalTravelTime.minimumDurationInMinutes',
       'summaryAdditionalTravelTime.maximumDurationInMinutes', 'sectionType',
       'section.stations', 'section.direction',
       'consequence.section.direction', 'consequence.description',
       'consequence.level', 'uicCode', 'stationCode', 'name', 'countryCode',
       'coordinate.lat', 'coordinate.lng'],
      dtype='object')

In [135]:
df_d3=df_d2.drop(['summaryAdditionalTravelTime.label','summaryAdditionalTravelTime.shortLabel'],axis=1)

In [138]:
df_d3=df_d3.drop(['section.stations'],axis=1)

In [140]:
df_d3.to_csv('disruptionsf.csv',index=False)